In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import statsmodels.api as sm

from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# The dataset

This dataset contains separated files for each car manufacturer. Let's try to work one at a time. Starting with the VW dataset.

In [ ]:
# Loading the data
vw_raw_data = pd.read_csv('/kaggle/input/used-car-dataset-ford-and-mercedes/vw.csv')
vw_raw_data.head()

In [ ]:
# Describe method
vw_raw_data.describe()

# About the VW dataset

* Has over 10 thousand observations, no NaN values, which is pretty good
* I Would like to see more features, but it keeps the model simple
* Let's visualize the data and build our regression models

# Data Visualization

In [ ]:
# Year and Price distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 6))
ax1.hist(vw_raw_data['year'], bins = 12, color = 'slateblue')
ax1.set_title('Year distribution', fontsize = 16, weight = 'bold')
ax1.set_xlabel('Year', fontsize = 14)
ax1.set_ylabel('Count', fontsize = 14)

ax2.hist(vw_raw_data['price'], bins = 12, color = 'midnightblue')
ax2.set_title('Price distribution', fontsize = 16, weight = 'bold')
ax2.set_xlabel('Price', fontsize = 14)
ax2.set_ylabel('Count', fontsize = 14)

plt.show()

In [ ]:
# Seaborn PairGrid of scatter plots
g = sns.PairGrid(vw_raw_data)
g.map(plt.scatter)

In [ ]:
# Seaborn PairGrid of scatter plots
sns.heatmap(vw_raw_data.corr(), annot=True)

# About the correlation matrix

The correlation matrix shows us that the price is correlated with all the variables. The less correlated variable is tax, and is still 0.48. Let's try and build a model then.

# Data preprocessing

We need to deal with the categorical columns creating dummies, let's check how many unique values they have


In [ ]:
# Categorical columns unique values
print(vw_raw_data['model'].unique(), vw_raw_data['transmission'].unique(), vw_raw_data['fuelType'].unique())

In [ ]:
# Creating dummies for those values
df = pd.get_dummies(vw_raw_data)
print(df.columns)
df.head()

# Building the models

In [ ]:
# Separating inputs and targets
targets = df['price']
inputs = df.drop(['price'], axis=1)

# Normalizing the data
scaler = StandardScaler()
scaler.fit(inputs)

inputs_scaled = scaler.transform(inputs)

# Splitting the data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(inputs_scaled, targets, test_size = 0.2)

In [ ]:
# Multiple Linear Regression with StatsModels
inputs_sm = sm.add_constant(inputs)
sm_model = sm.OLS(targets, inputs_sm)

# Summary table
print(sm_model.fit().summary())

In [ ]:
# Multiple Linear Regression with sklearn
reg = LinearRegression()
reg.fit(x_train, y_train)

# Cross validation
np.mean(cross_val_score(reg, x_train, y_train, cv=10, scoring='neg_mean_absolute_error'))

In [ ]:
alpha_ridge = []
error_ridge = []

for i in range(1500, 2000):
    alpha_ridge.append(i/10)
    ridge_cv = Ridge(alpha=(i/10.0))
    error_ridge.append(np.mean(cross_val_score(ridge_cv, x_train, y_train, cv=4, scoring='neg_mean_absolute_error')))

plt.plot(alpha_ridge, error_ridge)

df_err_ridge = pd.DataFrame(data = {'alpha': alpha_ridge, 'error': error_ridge})
ridge_alpha = df_err_ridge['alpha'][df_err_ridge['error'].idxmax()]

# Ridge Regression
ridge = Ridge(alpha=ridge_alpha)
ridge.fit(x_train, y_train)

In [ ]:
alpha_lasso = []
error_lasso = []

for i in range(50, 250):
    alpha_lasso.append(i/10)
    lasso_cv = Lasso(alpha=(i/10.0))
    error_lasso.append(np.mean(cross_val_score(lasso_cv, x_train, y_train, cv=4, scoring='neg_mean_absolute_error')))

plt.plot(alpha_lasso, error_lasso)

df_err_lasso = pd.DataFrame(data = {'alpha': alpha_lasso, 'error': error_lasso})
lasso_alpha = df_err_lasso['alpha'][df_err_lasso['error'].idxmax()]

# Lasso Regression
lasso = Lasso(alpha=lasso_alpha)
lasso.fit(x_train, y_train)

In [ ]:
# Testing the models

# OLS
tpred_reg = reg.predict(x_test)
# Ridge
tpred_ridge = ridge.predict(x_test)
# Lasso
tpred_lasso = lasso.predict(x_test)

print('OLS mae: ', mean_absolute_error(y_test, tpred_reg))
print('Ridge mae: ', mean_absolute_error(y_test, tpred_ridge))
print('Lasso mae: ', mean_absolute_error(y_test, tpred_lasso))